In [ ]:
import pandas as pd
import talib
import numpy as np

# Initial parameters
initial_balance = 100  # USD
leverage = 1000  # Leverage 1:1000
lot_size = 0.01  # Fixed Lot Size 0.01
spread_pips = 0.8  # Spread 0.8 pips
tp_pips = 30  # Take Profit 30 pips
sl_pips = 15  # Stop Loss 15 pips
pip_value = 0.2  # Value of 1 pip for lot size 0.01 in EURUSD (0.1 USD/pip)
max_orders = 3  # Maximum number of simultaneous orders
bollinger_period = 20  # Period for Bollinger Bands
bollinger_std = 2  # Number of standard deviations for Bollinger Bands

# Read CSV file
df = pd.read_csv(r"C:\Users\MongkolChut\Downloads\forex_trading\IUX\EURUSD.iux_M5_20250201_20250315.csv")

# Prepare data
df['open2'] = df['Open'].shift(2)
df['high2'] = df['High'].shift(2)
df['low2'] = df['Low'].shift(2)
df['close2'] = df['Close'].shift(2)
df['open1'] = df['Open'].shift(1)
df['high1'] = df['High'].shift(1)
df['low1'] = df['Low'].shift(1)
df['close1'] = df['Close'].shift(1)
df['rsi'] = talib.RSI(df['Close'], timeperiod=14)
df['rsi_1'] = df['rsi'].shift(1)
df['rsi_2'] = df['rsi'].shift(2)

# Calculate Bollinger Bands
df['bb_upper'], df['bb_middle'], df['bb_lower'] = talib.BBANDS(
    df['Close'], timeperiod=bollinger_period, nbdevup=bollinger_std, nbdevdn=bollinger_std, matype=0
)

# Calculate bid and ask prices
spread_value = spread_pips * 0.0001  # Spread in price terms (0.8 pips = 0.00008)
df['ask'] = df['Close'] + (spread_value / 2)  # Ask price
df['bid'] = df['Close'] - (spread_value / 2)  # Bid price

# Variables for results
balance = initial_balance
equity = [initial_balance]
trades = []
open_trades = []  # List to store open orders
total_rows = len(df)

# Function to calculate Maximum Drawdown
def calculate_max_drawdown(equity):
    equity = np.array(equity)
    peak = equity[0]
    max_drawdown = 0
    for value in equity:
        if value > peak:
            peak = value
        drawdown = (peak - value) / peak * 100
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

# Candlestick Pattern Functions (unchanged)
# Buy Patterns
def is_morning_star(df):
    first_candle_bearish = df['close2'] < df['open2']
    small_body_second_day = abs(df['close1'] - df['open1']) <= (df['high1'] - df['low1']) / 3
    third_candle_bullish = df['Close'] > df['Open']
    second_candle_gap_down = df['open1'] < df['close2']
    third_candle_closes_above_first_midpoint = df['Close'] > (df['open2'] + (df['close2'] - df['open2']) / 2)
    return first_candle_bearish & small_body_second_day & second_candle_gap_down & third_candle_bullish & third_candle_closes_above_first_midpoint

def is_bullish_abandoned_baby(df):
    first_candle_bearish = df['close2'] < df['open2']
    doji_second_day = (abs(df['close1'] - df['open1']) <= (df['high1'] - df['low1']) / 10)
    third_candle_bullish = df['Close'] > df['Open']
    second_candle_gap_down = (df['high1'] < df['low2'])
    third_candle_gap_up = (df['Low'] > df['high1'])
    return first_candle_bearish & doji_second_day & second_candle_gap_down & third_candle_bullish & third_candle_gap_up 

def is_three_inside_up(data):
    first_candle_bearish = data['close2'] < data['open2']
    second_candle_bearish = data['close1'] < data['open1']
    second_opens_lower = data['open1'] < data['close2']
    second_closes_in_first = data['close1'] > data['open2'] and data['close1'] < data['close2']
    third_candle_bullish = data['Close'] > data['Open']
    third_opens_above_second = data['Open'] > data['close1']
    third_closes_above_first = data['Close'] > data['close2']
    return (first_candle_bearish and second_candle_bearish and second_opens_lower and 
            second_closes_in_first and third_candle_bullish and third_opens_above_second and 
            third_closes_above_first)

def is_bullish_stick_sandwich(df):
    first_candle_bullish = df['close2'] > df['open2']
    second_candle_bearish = df['close1'] < df['open1']
    third_candle_bullish = (df['Close'] == df['close2']) & (df['Close'] > df['Open'])
    return first_candle_bullish & second_candle_bearish & third_candle_bullish

def is_bullish_marubozu(df):
    no_shadows = (df['Open'] == df['Low']) & (df['Close'] == df['High'])
    green_candle = df['Close'] > df['Open']
    return no_shadows & green_candle

def is_three_bullish_soldiers(df):
    return all([
        df['open2'] < df['close2'],  # first candle bullish
        df['open1'] < df['close1'],  # second candle bullish
        df['Open'] < df['Close'],    # third candle bullish
        df['close2'] < df['close1'], # higher closes
        df['close1'] < df['Close'],
        df['open1'] > df['close2'],  # opening within the body
        df['Open'] > df['close1'],
        df['open1'] < df['Open'],    # ascending opens
        df['open2'] < df['open1']
    ])

# Sell Patterns
def is_bearish_evening_star(df):
    first_candle_bullish = df['close2'] > df['open2']
    small_body_second_day = abs(df['close1'] - df['open1']) <= (df['high1'] - df['low1']) / 3
    third_candle_bearish = df['Close'] < df['Open']
    second_candle_gap_up = df['open1'] > df['close2']
    third_candle_closes_below_first_midpoint = df['Close'] < (df['open2'] + (df['close2'] - df['open2']) / 2)
    return first_candle_bullish & small_body_second_day & second_candle_gap_up & third_candle_bearish & third_candle_closes_below_first_midpoint

def is_bearish_abandoned_baby(df):
    first_candle_bullish = df['close2'] > df['open2']
    doji_second_day = (abs(df['close1'] - df['open1']) <= (df['high1'] - df['low1']) / 10)
    third_candle_bearish = df['Close'] < df['Open']
    second_candle_gap_up = (df['low1'] > df['high2'])
    third_candle_gap_down = (df['High'] < df['low1'])
    return first_candle_bullish & doji_second_day & second_candle_gap_up & third_candle_bearish & third_candle_gap_down  

def is_three_inside_down(data):
    first_candle_bullish = data['close2'] > data['open2']
    second_candle_bullish = data['close1'] > data['open1']
    second_opens_higher = data['open1'] > data['close2']
    second_closes_in_first = data['close1'] < data['close2'] and data['close1'] > data['open2']
    third_candle_bearish = data['Close'] < data['Open']
    third_opens_below_second = data['Open'] < data['close1']
    third_closes_below_first = data['Close'] < data['close2']
    return (first_candle_bullish and second_candle_bullish and second_opens_higher and 
            second_closes_in_first and third_candle_bearish and third_opens_below_second and
            third_closes_below_first)

def is_bearish_stick_sandwich(df):
    first_candle_bearish = df['close2'] < df['open2']
    second_candle_bullish = df['close1'] > df['open1']
    third_candle_bearish = (df['Close'] == df['close2']) & (df['Close'] < df['Open'])
    return first_candle_bearish & second_candle_bullish & third_candle_bearish

def is_bearish_marubozu(df):
    no_shadows = (df['Open'] == df['High']) & (df['Close'] == df['Low'])
    red_candle = df['Close'] < df['Open']
    return no_shadows & red_candle

def is_three_black_crows(df):
    return all([
        df['open2'] > df['close2'],  # first candle bearish
        df['open1'] > df['close1'],  # second candle bearish
        df['Open'] > df['Close'],    # third candle bearish
        df['close2'] > df['close1'], # lower closes
        df['close1'] > df['Close'],
        df['open1'] < df['close2'],  # opening within the body
        df['Open'] < df['close1'],
        df['open1'] > df['Open'],    # descending opens
        df['open2'] > df['open1']
    ])

def is_bullish_engulfing(df):
    first_candle_bearish = df['close1'] < df['open1']
    second_candle_bullish = df['Close'] > df['Open']
    engulfing_body = (df['Open'] <= df['close1']) & (df['Close'] >= df['open1'])
    return first_candle_bearish & second_candle_bullish & engulfing_body

def is_bearish_engulfing(df):
    first_candle_bullish = df['close1'] > df['open1']
    second_candle_bearish = df['Close'] < df['Open']
    engulfing_body = (df['Open'] >= df['close1']) & (df['Close'] <= df['open1'])
    return first_candle_bullish & second_candle_bearish & engulfing_body

# Simulate trading
for i in range(15, total_rows):
    candle = df.iloc[i]

    # Skip if previous candle has no range
    if candle['high1'] - candle['low1'] == 0:
        continue

    # Check for closing open orders
    closed_trades = []
    for trade in open_trades[:]:
        current_ask = candle['ask']
        current_bid = candle['bid']
        profit_pips = 0
        trade_closed = False

        if trade['type'] == 'buy':
            current_price = current_bid  # Exit buy at bid price
            if current_price >= trade['tp_price']:
                profit_pips = tp_pips
                trade_closed = True
            elif current_price <= trade['sl_price']:
                profit_pips = -sl_pips
                trade_closed = True
        elif trade['type'] == 'sell':
            current_price = current_ask  # Exit sell at ask price
            if current_price <= trade['tp_price']:
                profit_pips = tp_pips
                trade_closed = True
            elif current_price >= trade['sl_price']:
                profit_pips = -sl_pips
                trade_closed = True

        if trade_closed:
            profit = profit_pips * pip_value
            balance += profit
            equity.append(balance)
            trades.append({
                'type': trade['type'],
                'entry': trade['entry_price'],
                'exit': current_price,
                'profit_pips': profit_pips,
                'profit_usd': profit,
                'spread_cost': spread_pips * pip_value  # For reporting
            })
            closed_trades.append(trade)

    # Remove closed trades
    for trade in closed_trades:
        open_trades.remove(trade)

    # Check for new signals (if fewer than max_orders)
    if len(open_trades) < max_orders:
        # Check if an order is already open at this index
        has_trade_at_index = any(trade['index'] == i for trade in open_trades)
        if not has_trade_at_index and not pd.isna(candle['bb_upper']):
            # Bullish Patterns
            bullish_patterns = (
                is_morning_star(candle) or
                is_bullish_abandoned_baby(candle) or
                is_three_inside_up(candle) or
                is_bullish_stick_sandwich(candle) or
                is_bullish_marubozu(candle) or
                is_three_bullish_soldiers(candle) or
                is_bullish_engulfing(candle)
            )
            # Bearish Patterns
            bearish_patterns = (
                is_bearish_evening_star(candle) or
                is_bearish_abandoned_baby(candle) or
                is_three_inside_down(candle) or
                is_bearish_stick_sandwich(candle) or
                is_bearish_marubozu(candle) or
                is_three_black_crows(candle) or
                is_bearish_engulfing(candle)
            )

            # Bollinger Bands Conditions
            price_cross_above_lower_bb = (candle['close1'] <= candle['bb_lower'] and candle['Close'] > candle['bb_lower'])
            price_cross_below_upper_bb = (candle['close1'] >= candle['bb_upper'] and candle['Close'] < candle['bb_upper'])

            # Trading Logic with RSI, Candlestick Patterns, and Bollinger Bands
            if (candle['rsi'] > 80 and is_morning_star(candle)) or (bullish_patterns and price_cross_above_lower_bb):
                entry_price = candle['ask']  # Enter buy at ask price
                equity.append(balance)  # Record equity before trade
                open_trades.append({
                    'type': 'buy',
                    'entry_price': entry_price,
                    'sl_price': entry_price - sl_pips * 0.0001,
                    'tp_price': entry_price + tp_pips * 0.0001,
                    'index': i
                })
            elif (candle['rsi'] < 20 and is_bearish_evening_star(candle)) or (bearish_patterns and price_cross_below_upper_bb):
                entry_price = candle['bid']  # Enter sell at bid price
                equity.append(balance)  # Record equity before trade
                open_trades.append({
                    'type': 'sell',
                    'entry_price': entry_price,
                    'sl_price': entry_price + sl_pips * 0.0001,
                    'tp_price': entry_price - tp_pips * 0.0001,
                    'index': i
                })
            else:
                # Candlestick Patterns without RSI but with Bollinger Bands
                if bullish_patterns and price_cross_above_lower_bb:
                    entry_price = candle['ask']  # Enter buy at ask price
                    equity.append(balance)  # Record equity before trade
                    open_trades.append({
                        'type': 'buy',
                        'entry_price': entry_price,
                        'sl_price': entry_price - sl_pips * 0.0001,
                        'tp_price': entry_price + tp_pips * 0.0001,
                        'index': i
                    })
                elif bearish_patterns and price_cross_below_upper_bb:
                    entry_price = candle['bid']  # Enter sell at bid price
                    equity.append(balance)  # Record equity before trade
                    open_trades.append({
                        'type': 'sell',
                        'entry_price': entry_price,
                        'sl_price': entry_price + sl_pips * 0.0001,
                        'tp_price': entry_price - tp_pips * 0.0001,
                        'index': i
                    })

# Close remaining open orders
for trade in open_trades:
    last_ask = df.iloc[-1]['ask']
    last_bid = df.iloc[-1]['bid']
    profit_pips = 0
    exit_price = 0
    if trade['type'] == 'buy':
        exit_price = last_bid  # Exit buy at bid price
        profit_pips = (exit_price - trade['entry_price']) / 0.0001
    elif trade['type'] == 'sell':
        exit_price = last_ask  # Exit sell at ask price
        profit_pips = (trade['entry_price'] - exit_price) / 0.0001
    profit = profit_pips * pip_value
    balance += profit
    equity.append(balance)
    trades.append({
        'type': trade['type'],
        'entry': trade['entry_price'],
        'exit': exit_price,
        'profit_pips': profit_pips,
        'profit_usd': profit,
        'spread_cost': spread_pips * pip_value  # For reporting
    })

# Calculate results
total_trades = len(trades)
win_trades = sum(1 for trade in trades if trade['profit_pips'] > 0)
win_rate = (win_trades / total_trades * 100) if total_trades > 0 else 0
final_balance = balance
total_profit = final_balance - initial_balance
max_drawdown = calculate_max_drawdown(equity)

# Display results
print(f"Initial Balance: ${initial_balance:.2f}")
print(f"Final Balance: ${final_balance:.2f}")
print(f"Total Profit: ${total_profit:.2f}")
print(f"Total Trades: {total_trades}")
print(f"Win Trades: {win_trades}")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Maximum Drawdown: {max_drawdown:.2f}%")

# Display trade details
for i, trade in enumerate(trades, 1):
    print(f"Trade {i}: {trade['type'].upper()} | Entry: {trade['entry']:.5f} | Exit: {trade['exit']:.5f} | "
          f"Profit: {trade['profit_pips']:.2f} pips | ${trade['profit_usd']:.2f} | Spread Cost: ${trade['spread_cost']:.2f}")